In [6]:
# Modes of regulation that turn on under Hh conditions, relative to LI GF

# GOAL: We want to visualize each quantitative ATAC priors and compare networks resulting from different
# choices of geneset and peakset cutoffs (e.g., log2 fold-change and FDR), for each network (e.g., SI gene regulation
# in macrophages).  We will use the "multiple_network" functionality to do this, and use networks linked to heatmaps
# ("Lexpression" functionality) so that we can color nodes according to gene expression 

# Each tupple corresponds to (geneset cutoff, peakset cutoff)
cutComb = ('FC1_FDR10','FC1_FDR10')
# ('FC0p58_FDR25','FC0p58_FDR25')
# 
#                 ('FC1_FDR10','FC0p58_FDR25'),
#                 ('FC0p58_FDR25','FC0p58_FDR25'),
#                 ('FC0p58_FDR25','FC1_FDR10')]
# NOTE: Script below is HARD-coded to only visualize the six (LI) or five (SI) networks

directory = "/Users/emiraldi/erm/Shared/DCproject/DCquantAtacNetworks" # note "try" statement below, will also check current directory

# Location of each network, and note we will use .replace() to insert the gene and peakset cutoffs of interest
# for "GENE_CUTOFF" and "PEAK_CUTOFF" in the string below
netBase = "DC_genesets_GENE_CUTOFF/DCs_max4_body_bp10000/DCnets_Hh/DCnets_Hh_pPEAK_CUTOFF_rawp0001_hyg0001"
geneCutoff = cutComb[0]
peakCutoff = cutComb[1]
netPath = netBase.replace('GENE_CUTOFF',geneCutoff).replace('PEAK_CUTOFF',peakCutoff)

# The starting conditions for each of the networks, a list of tuples.  Tuple entries are:
# 0. network file name (column format) (as found in directory + netBase with cutoffs inserted
# 1. column of the expression matrix that you want the nodes to be colored by
# 2. network title, to which we'll add the gene and peak cutoffs
# 3. subselect the list of relevant genes -- NOT USED for this code and can be left out
networkInits = [('CD11bmCD103m_Hh_sp.tsv','Hh_v_GF_CD11b-CD103-','Hh UP (Red), LI CD11b-CD103-','GeneSets/GF_LI_CD11bmCD103m_v_Hh_LI_CD11bmCD103m_FC1_FDR10_down.txt'),
	('CD103p_Hh_sp.tsv','Hh_v_GF_CD11b-CD103+','Hh UP (Red), LI CD11b-CD103+','GeneSets/GF_LI_CD11bmCD103p_v_Hh_LI_CD11bmCD103p_FC1_FDR10_down.txt'),
	('CD11bp_Hh_sp.tsv','Hh_v_GF_CD11b+CD103-','Hh UP (Red), LI CD11b+CD103-','GeneSets/GF_LI_CD11bpCD103m_v_Hh_LI_CD11bpCD103m_FC1_FDR10_down.txt'),
	('CD11bpCD103p_Hh_sp.tsv','Hh_v_GF_CD11b+CD103+','Hh UP (Red), LI CD11b+CD103+','GeneSets/GF_LI_CD11bpCD103p_v_Hh_LI_CD11bpCD103p_FC1_FDR10_down.txt'),
	('mphage_Hh_sp.tsv','Hh_v_GF_Macrophage','Hh UP (Red), LI Macrophage','GeneSets/GF_LI_Macrophage_v_Hh_LI_Macrophage_FC1_FDR10_down.txt'),
	('monocyte_Hh_sp.tsv','Hh_v_GF_Monocyte','Hh UP (Red), Monocyte','GeneSets/GF_LI_Monocyte_v_Hh_LI_Monocyte_FC1_FDR10_down.txt')]

expressionFile = "Heatmaps/Microb_DC_ILC3_log2FCs_all.txt"

tfFocus = 0 # If 1, automatically applies the "trim" function, so we can focus on TFs
    # If 0, does not!

threshhold = .2 # for edge weights in graph

clim = 1.5 # absolute value color threshhold on edge color

In [7]:
# Uncomment to run without install (in binder, for example)
import sys
if ".." not in sys.path:
    sys.path.append("..")
from jp_gene_viz import dNetwork
dNetwork.load_javascript_support()
from jp_gene_viz import multiple_network
from jp_gene_viz import LExpression
LExpression.load_javascript_support()
# Part 1 of a hacky way to set color threshhold
from numpy import array
minclr = array([   255.,  0,    0.])
minvalue = -clim
zeroclr = array([255.0000 , 130.4000  , 60.0000]) # [   166.,    86.,    40.]) #127.5, 63.75, 0 ])
maxvalue = clim
maxclr = array([ 0.,    1.7*127.5,    0.])


# Currently, we are not interested in visualizing motifs, and would need to find these files, if we did
# from jp_gene_viz import motif_data
# C = motif_data.MotifCollection()
# # for extra safety "rU" reads with universal line ending support
# C.read_meme_file(open(directory + '/' + "mm9_em.meme", "rU"))
# C.read_meme_file(open(directory + '/' + "hg19_em.meme", "rU"))

In [8]:
networkList = list()  # this list contains heatmap-linked network objects

for networkInit in networkInits:
    networkFile = networkInit[0]
    curr = LExpression.LinkedExpressionNetwork()
    try:
        curr.load_network(directory + '/' + netPath + '/' + networkFile)  
    except AssertionError:
        directory = "."
        curr.load_network(directory + '/' + netPath + '/' + networkFile)
    networkList.append(curr)


AssertionError: 

In [ ]:
# visualize the networks -- HARD CODED for 5 networks:
M = multiple_network.MultipleNetworks(
    [[networkList[0], networkList[1]],
    [networkList[2], networkList[3]],
    [networkList[4]]])
M.svg_width = 500
M.show()      

In [ ]:
# Set network preferences
count = 0
for curr in networkList:
    networkInit = networkInits[count]
    # get title information + curr column for shading of figures
    currCol = networkInit[1].lower()
    titleBase = networkInit[2]
    titleInf = titleBase + ' (gCut: ' + geneCutoff.replace('_',' ') + ', pCut: ' + peakCutoff.replace('_',' ')
    
    # set threshold
    curr.network.threshhold_slider.value = threshhold
    curr.network.apply_click(None)
    if tfFocus:
        # focus on TF core    
        curr.network.tf_only_click(None)
        curr.network.layout_click(None)  
        
    # set title
    curr.network.title_html.value = titleInf

    # add labels
    curr.network.labels_button.value=True  
    curr.network.draw_click(None)

    # Load heatmap
    curr.load_heatmap(directory + '/' + expressionFile)
    # color nodes according to a sample column in the gene expression matrix
    curr.gene_click(None)    
    curr.expression.col = currCol    
    curr.condition_click(None)
    
        #     # Attach the motif collection populated above:
        #     N.motif_collection = C
        #     net_with_motifs
    
    # !!!!! for some reason this doesn't work, tried curr.display... w/o curr.network.display...
    # Part 2: a hacky way to set min and max on the heatmap and heatmap-colored nodes
    curr.network.display_graph.get_edge_color_interpolator()
    curr.network.display_graph._edge_color_interpolator.minclr = minclr
    curr.network.display_graph._edge_color_interpolator.maxclr = maxclr
    curr.network.display_graph._edge_color_interpolator.breakpoints = \
        [(minvalue, minclr),
         (0, zeroclr),
         (maxvalue, maxclr)]
    curr.network.draw_click(None)
    count += 1
